52100674 - TRẦN THỊ VẸN

1) Present the basics of the Convolutional Neural Networks model

2) Present the architecture of the Residual Networks model and its variations.

3) Apply ResNet in image recognition and compare with conventional CNN models

**1) Present the basics of the Convolutional Neural Networks model**

Mạng CNN là một tập hợp các lớp Convolution chồng lên nhau và sử dụng các hàm nonlinear activation như ReLU và tanh để kích hoạt các trọng số trong các node. Mỗi một lớp sau khi thông qua các hàm kích hoạt sẽ tạo ra các thông tin trừu tượng hơn cho các lớp tiếp theo.

Mỗi một lớp sau khi thông qua các hàm kích hoạt sẽ tạo ra các thông tin trừu tượng hơn cho các lớp tiếp theo. Trong mô hình mạng truyền ngược (feedforward neural network) thì mỗi neural đầu vào (input node) cho mỗi neural đầu ra trong các lớp tiếp theo.

+ Convolutional layers (lớp tích chập): Áp dụng các bộ lọc (kernels) để trích xuất các đặc trưng từ dữ liệu đầu vào thông qua việc tích chập.
+ Lớp tổng hợp (pooling layer): Giảm kích thước không gian của biểu diễn đặc trưng bằng cách thực hiện các phép tổng hợp như max pooling hoặc average pooling.
+ Activation functions (hàm kích hoạt): Thông thường sử dụng hàm kích hoạt phi tuyến tính như ReLU để thêm tính phi tuyến tính vào mô hình.
+ Fully connected layers (lớp kết nối đầy đủ): Lớp cuối cùng của mạng, thường được sử dụng để chuyển từ biểu diễn đặc trưng sang biểu diễn phân loại.

**Cách chọn tham số cho CNN**
+ Số các convolution layer: càng nhiều các convolution layer thì performance càng được cải thiện. Sau khoảng 3 hoặc 4 layer, các tác động được giảm một cách đáng kể
+ Filter size: thường filter theo size 5×5 hoặc 3×3
+ Pooling size: thường là 2×2 hoặc 4×4 cho ảnh đầu vào lớn
+ Cách cuối cùng là thực hiện nhiều lần việc train test để chọn ra được param tốt nhất.

**2) Present the architecture of the Residual Networks model and its variations**

ResNet là một loại mô hình CNN nổi tiếng được giới thiệu bởi Microsoft Research.

Kiến trúc chính của ResNet là sự thêm vào các đơn vị dư (residual units) để giải quyết vấn đề độ sâu của mạng. Các đơn vị dư cho phép thông tin truyền qua mạng mà không bị giảm độ chính xác do sự mất mát thông tin.

ResNet gần như tương tự với các mạng gồm có convolution, pooling, activation và fully-connected layer. Ảnh bên trên hiển thị khối dư được sử dụng trong mạng. Xuất hiện một mũi tên cong xuất phát từ đầu và kết thúc tại cuối khối dư. Hay nói cách khác là sẽ bổ sung Input X vào đầu ra của layer, hay chính là phép cộng mà ta thấy trong hình minh họa, việc này sẽ chống lại việc đạo hàm bằng 0, do vẫn còn cộng thêm X. Với H(x) là giá trị dự đoán, F(x) là giá trị thật (nhãn), chúng ta muốn H(x) bằng hoặc xấp xỉ F(x). Việc F(x) có được từ x như sau:

X->weight1-> ReLU -> weight2

Giá trị H(x) có được bằng cách:

F(x) + x -> ReLU

**Một số biến thể của ResNet bao gồm:**
+ ResNet-18, ResNet-34: Các phiên bản nhẹ của ResNet với số lượng lớp tương ứng là 18 và 34.
+ ResNet-50, ResNet-101, ResNet-152: Các phiên bản nặng hơn của ResNet với số lượng lớp tương ứng là 50, 101 và 152.

**3) Apply ResNet in image recognition and compare with conventional CNN models**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

# Chuẩn bị dữ liệu
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

100%|██████████| 170498071/170498071 [00:05<00:00, 28657465.39it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# Mô hình CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# # Mô hình ResNet
# class ResNet(nn.Module):
#     def __init__(self):
#         super(ResNet, self).__init__()
#         self.resnet = resnet18(pretrained=False, num_classes=10)

#     def forward(self, x):
#         return self.resnet(x)

In [5]:
def train(model, criterion, optimizer, trainloader, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

In [6]:
def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {correct/total}")

In [7]:
cnn_model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)
train(cnn_model, criterion, optimizer, trainloader)
test(cnn_model, testloader)

Epoch 1, Loss: 1.20242922373178
Epoch 2, Loss: 0.8062067246368431
Epoch 3, Loss: 0.5946559427223828
Epoch 4, Loss: 0.41221145488993904
Epoch 5, Loss: 0.25284707510363613
Accuracy: 0.7422


In [ ]:
# resnet_model = ResNet()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
# train(resnet_model, criterion, optimizer, trainloader)
# test(resnet_model, testloader)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

# Chuẩn bị dữ liệu
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False)

# Mô hình ResNet
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.resnet = resnet18(pretrained=False, num_classes=10)

    def forward(self, x):
        return self.resnet(x)

def train(model, criterion, optimizer, trainloader, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {correct/total}")

# Huấn luyện và đánh giá mô hình ResNet
resnet_model = ResNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.01)  # Thử tăng learning rate
train(resnet_model, criterion, optimizer, trainloader, epochs=5)  # Giảm số lượng epoch
test(resnet_model, testloader)

Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 1.72664173286438
